# Óbitos por câncer de pulmão

Iremos analisar o número de mortes por câncer de pulmão em uma população composta por médicos britânicos, segundo sua faixa etária e o consumo médio diário de cigarros. 

Ao longo do documento realizaremos algumas análises e responderemos as seguintes questões:

- Quais são as variáveis resposta e explicativas envolvidas no estudo.
- Qual o modelo proposto.
- Ajuste do modelo e estatísticas utilizadas para isso.
- Descrição sobre a significância individual dos parâmetros. Estatísticas utilizadas para tais conclusões?
- Equação estimada pelo modelo.
- Interpretação dos parâmetros do modelo.

In [1]:
#Armazenando os dados
obitos <- c(0,3,0,3,0,1,3,3,7,29,41,45,3,16,36,11)
pop <- c(33679,21131,10599,4495,6002,4396,2813,1664,34414,25429,13271,4765,5881,6493,3466,769)
tabela <- expand.grid(FaixaEtaria = c('40-49','50-59','60-69','70-80'),Cigarros = c('0','1-9','10-30','30+'))

#Concatenando em uma tabela
dados <- cbind(tabela,obitos,pop)

#Mostrando parte dos dados
head(dados)

FaixaEtaria,Cigarros,obitos,pop
40-49,0,0,33679
50-59,0,3,21131
60-69,0,0,10599
70-80,0,3,4495
40-49,1-9,0,6002
50-59,1-9,1,4396


## Variável resposta e explicativas

Note que o objetivo deste documento é analisar o número de óbitos, logo, essa será a nossa variável resposta, i.e, o que desejamos estimar. A faixa etaria, número de cigarros diário e população serão nossas variáveis explicativas.

## Modelo estatístico

Assim, a partir das variáveis explicativas tentaremos estimar os óbitos por meio de um modelo linear generalizado (mlg ou glm em inglês) que será definido da seguinte maneira:

- Componente aleatório: Poisson 
- Função de ligação: Log

A explicação da utilização desses componentes é que a distribuição Poisson é geralmente adequada quanto a variável resposta avalia as contagens e a função de ligação log é usada pois em geral ela possui bom desempenho e poder descritivo quando usado com a Poisson. Deve-se lembrar também que tal ligação é conhecida por ser canônica e possui características estatísticas interessantes na modelagem.

In [2]:
#Criação do modelo utilizando glm
mod <- glm(obitos ~ offset(log(pop)) + FaixaEtaria + Cigarros, family = poisson,data=dados)

In [3]:
#Avaliação do modelo
summary(mod)


Call:
glm(formula = obitos ~ offset(log(pop)) + FaixaEtaria + Cigarros, 
    family = poisson, data = dados)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.04919  -0.74414  -0.04008   0.47261   1.21623  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -11.4241     0.5094 -22.424  < 2e-16 ***
FaixaEtaria50-59   1.7693     0.3473   5.095 3.49e-07 ***
FaixaEtaria60-69   2.8973     0.3357   8.630  < 2e-16 ***
FaixaEtaria70-80   3.7915     0.3409  11.121  < 2e-16 ***
Cigarros1-9        1.4090     0.5566   2.531   0.0114 *  
Cigarros10-30      2.8662     0.4182   6.854 7.20e-12 ***
Cigarros30+        3.7581     0.4269   8.803  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 472.596  on 15  degrees of freedom
Residual deviance:  11.912  on  9  degrees of freedom
AIC: 77.391

Number of Fisher Scoring iterations: 5


## Ajuste do modelo

Analisando a saída da função summary vemos que o desvio residual é relativamente baixo e comparando-o com o desvio nulo vemos que há uma grande diferença entre eles, logo, pode-se inferir que a utilização de outras variáveis (tirando a offset) auxiliou no modelagem. Portanto, acreditamos que nosso modelo está bem ajustado.

## Significância dos parâmetros

Analisando o p-value calculado em cima do z-value verificamos que as suas probabilidades são bem pequenas, isso signigica que a chance da hipótese nula ser aceita é muito pequena, ou seja, mesmo que utilizassemos um coeficiente de significância bem pequeno, ainda rejeitaríamos a hipótese nula. Como tal hipótese considera que os beta's são iguais a 0 e nós a rejeitamos, podemos concluir que nenhum dos coeficientes calculados devem ser desconsiderados, logo todos possuem uma alta significância no modelo.

## Equação estimada

Dado a saída da função summary, verificamos que a equação proposta pelo modelo possui a seguinte forma

Y = 1.7693X1 + 2.8973X2 + 3.7915X3 + 1.4090X4 + 2.8662X5 + 3.7581X6 -11.4241

onde

Y = variável resposta (número de óbitos) <br>
X1 = Faixa etária 50-59<br>
X2 = Faixa etária 60-69<br>
X3 = Faixa etária 70-80<br>
X4 = Cigarros 1-9<br>
X5 = Cigarros 10-30<br>
X6 = Cigarros 30+<br>


## Interpretação dos parâmetros